![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [51]:
-- highest_donation_assignments
WITH assignment_donor AS (
	SELECT 
		assignment_id,
		donor_type,
		ROUND(SUM(amount), 2) AS rounded_total_donation_amount
	FROM donations 
	LEFT JOIN donors d
		USING (donor_id)
	GROUP BY 
		assignment_id, 
		donor_type
)

SELECT 
	assignment_name,
	region,
	rounded_total_donation_amount,
	donor_type
FROM assignments a
JOIN assignment_donor ad
	USING (assignment_id)
WHERE rounded_total_donation_amount IS NOT NULL
ORDER BY rounded_total_donation_amount DESC
LIMIT 5

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [52]:
-- top_regional_impact_assignments
WITH ranked AS (
	SELECT 
		assignment_name,
		assignment_id, 
		region, 
		impact_score,
		ROW_NUMBER() OVER (PARTITION BY region ORDER BY impact_score DESC) as ranking
	FROM assignments
),
donation_count AS (
	SELECT 
		assignment_id, 
		COUNT(donation_id) as num_total_donations
	FROM donations 
	GROUP BY assignment_id
)

SELECT 
	assignment_name,
	region,
	impact_score,
	num_total_donations
FROM ranked
JOIN donation_count 
	USING (assignment_id)
WHERE ranking = 1
ORDER BY region ASC;


,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_3764,West,9.99,1


In [53]:
SELECT * FROM donations WHERE assignment_id IN ('3764', '2794')

,donation_id,donor_id,amount,donation_date,assignment_id
0,2120,910,73.36,2022-10-16 00:00:00+00:00,2794
1,4039,4053,716.72,2020-05-27 00:00:00+00:00,3764
2,4464,4840,618.22,2021-04-29 00:00:00+00:00,2794
